<a href="https://colab.research.google.com/github/keshavga1209/yeagerists/blob/flask/Gaurav/Emotion_Detection_Models/jupyter_nb/arch_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import PIL
import pathlib
import tensorflow as tf

In [2]:
data_dir = "/content/drive/MyDrive/hack_the_tank/Feet"
data_dir = pathlib.Path(data_dir)
img_height = 256
img_width = 256
# batch_size = 17640

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)


1215


In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=5,
  shuffle=True,
  subset="training",
  image_size=(img_height, img_width),
  batch_size=32)

Found 1217 files belonging to 3 classes.
Using 974 files for training.


In [6]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=5,
  shuffle=True,
  subset="validation",
  image_size=(img_height, img_width),
  batch_size=32)

Found 1217 files belonging to 3 classes.
Using 243 files for validation.


In [7]:
classes = train_ds.class_names
print(classes)

['FLAT', 'HIGH', 'NORMAL']


In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
model = tf.keras.Sequential([
                             tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
                             tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(256, 256, 3)),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Conv2D(32, (5, 5), activation='relu'),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Conv2D(32, (5, 5), activation='relu'),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation="relu"),
                             tf.keras.layers.Dense(128, activation="relu"),
                             tf.keras.layers.Dense(64, activation="relu"),
                             tf.keras.layers.Dense(32, activation="relu"),
                             tf.keras.layers.Dense(3)
])

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>0.98):
        print("\n 98% acc reached")
        self.model.stop_training = True

In [12]:
model.fit(
  train,
  validation_data=validation,
  epochs=50,
  callbacks=[CustomCallbacks()]
)

Epoch 1/50
31/31 [==============================] - 28s 377ms/step - loss: 0.9463 - accuracy: 0.7361 - val_loss: 0.8369 - val_accuracy: 0.7325
Epoch 2/50
31/31 [==============================] - 2s 60ms/step - loss: 0.7689 - accuracy: 0.7392 - val_loss: 0.7342 - val_accuracy: 0.7325
Epoch 3/50
31/31 [==============================] - 2s 61ms/step - loss: 0.7113 - accuracy: 0.7382 - val_loss: 0.6964 - val_accuracy: 0.7325
Epoch 4/50
31/31 [==============================] - 2s 59ms/step - loss: 0.6383 - accuracy: 0.7515 - val_loss: 0.6214 - val_accuracy: 0.7695
Epoch 5/50
31/31 [==============================] - 2s 59ms/step - loss: 0.5580 - accuracy: 0.7875 - val_loss: 0.6122 - val_accuracy: 0.7819
Epoch 6/50
31/31 [==============================] - 2s 59ms/step - loss: 0.5113 - accuracy: 0.8018 - val_loss: 0.5668 - val_accuracy: 0.7860
Epoch 7/50
31/31 [==============================] - 2s 59ms/step - loss: 0.3975 - accuracy: 0.8388 - val_loss: 0.6279 - val_accuracy: 0.7737
Epoch 8/50


In [13]:
# import joblib

In [14]:
# joblib.dump(model, '/content/drive/MyDrive/hack_the_tank/model1_simple.pkl')

In [15]:
model.save("/content/drive/MyDrive/hack_the_tank/model50_1")

In [16]:
# model = joblib.load("/content/model.pkl")
# print(model(np.random.random((1,5))))

In [17]:
# def test():

In [25]:
for f in os.listdir('/content/drive/MyDrive/hack_the_tank/test_dummy'):
    imgpath = os.path.join('/content/drive/MyDrive/hack_the_tank/test_dummy',f)
    img = keras.utils.load_img(
        imgpath, target_size=(256, 256)
    )
    img_array = keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    score = np.array(tf.nn.softmax(predictions[0]))
    print(predictions)
    print(imgpath, score)

1/1 [==============================] - 0s 20ms/step
[[-16.00083    -3.3058681  15.752536 ]]
/content/drive/MyDrive/hack_the_tank/test_dummy/flat2.jpeg [1.6206491e-14 5.2849485e-09 1.0000000e+00]
1/1 [==============================] - 0s 22ms/step
[[ 31.18276  -19.586487 -12.88805 ]]
/content/drive/MyDrive/hack_the_tank/test_dummy/flat4.jpeg [1.000000e+00 8.937074e-23 7.249217e-20]
1/1 [==============================] - 0s 19ms/step
[[ 22.957922 -10.385742 -21.290129]]
/content/drive/MyDrive/hack_the_tank/test_dummy/Flatfoot5.jpg [1.0000000e+00 3.3039223e-15 6.0717765e-20]
1/1 [==============================] - 0s 20ms/step
[[ 3.4571145 -3.015052  -0.7726442]]
/content/drive/MyDrive/hack_the_tank/test_dummy/High_arch.webp [0.98415333 0.00152138 0.01432524]
1/1 [==============================] - 0s 20ms/step
[[14.878993 -8.692794 -7.833652]]
/content/drive/MyDrive/hack_the_tank/test_dummy/arch_foot.jpg [1.0000000e+00 5.7929932e-11 1.3678038e-10]
1/1 [==============================] - 0s 

In [1]:
!pip freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
astropy==5.2.1
astunparse==1.6.3
attrs==22.2.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
bokeh==2.4.3
branca==0.6.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==3.0.4
charset-normalizer==2.0.12
chex==0.1.6
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==39.0.2
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.33
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.0.5
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
distributed==2022.12.1
dlib==19.24.0
dm-tr